In [1]:
import pyspark
pyspark.__version__

'3.5.1'

In [2]:
from pyspark.sql import SparkSession

In [3]:
# Create SparkSession object
spark = SparkSession.builder.master("local[1]").appName('test').getOrCreate()

24/07/28 06:32:45 WARN Utils: Your hostname, codespaces-a6e710 resolves to a loopback address: 127.0.0.1; using 10.0.2.199 instead (on interface eth0)
24/07/28 06:32:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/28 06:32:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Terminate the cluster
spark.stop()